# SVD

## 1. Install & Import Packages

In [52]:
import os
import pandas as pd

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import tensorflow as tf
# import tensorflow_recommenders as tfrs

from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, Callback
from tensorflow.keras.utils import plot_model
import glob

In [3]:
def get_path(filename):
  return f'drive/MyDrive/Colab Notebooks/Data/{filename}'

glob.glob("drive/MyDrive/Colab Notebooks/Data/*")

[]

In [5]:
df_train = pd.read_json(get_path('train_all_meta_v2.json'))
df_test = pd.read_json(get_path('test_all_meta_v2.json'))

In [6]:
from scipy.sparse.linalg import svds

In [ ]:
pivoted = df_train.pivot_table('rating_per_user', 'userID', 'wine_id', aggfunc='max')

In [ ]:
df_pivot = pivoted.fillna(0)
df_pivot

wine_id,10,19,20,41,46,91,93,114,115,116,127,128,130,135,138,183,198,204,255,271,287,288,296,315,342,344,375,423,433,450,451,452,453,455,457,460,462,473,510,531,...,8003932,8010383,8018306,8018733,8057324,8057883,8075591,8075594,8083851,8095543,8103001,8125365,8127729,8139168,8143180,8152983,8168002,8176779,8203097,8205857,8208422,8226290,8257837,8295247,8295713,8305831,8316340,8331658,8332127,8341431,8342417,8346518,8346599,8357660,8380143,8380239,8407279,8436727,8546714,8564957
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46021938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46231365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46356698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix = df_pivot.values
user_ratings_mean = np.mean(matrix, axis = 1)
# R_user_mean : 사용자-와인에 대해 사용자 평균 평점을 뺀 것.
matrix_tag_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix_tag_mean.shape

(6343, 50852)

In [ ]:
matrix_tag_mean

array([[-0.0123004 , -0.0123004 , -0.0123004 , ..., -0.0123004 ,
        -0.0123004 , -0.0123004 ],
       [-0.0003638 , -0.0003638 , -0.0003638 , ..., -0.0003638 ,
        -0.0003638 , -0.0003638 ],
       [-0.0007571 , -0.0007571 , -0.0007571 , ..., -0.0007571 ,
        -0.0007571 , -0.0007571 ],
       ...,
       [-0.00026548, -0.00026548, -0.00026548, ..., -0.00026548,
        -0.00026548, -0.00026548],
       [-0.01607606, -0.01607606, -0.01607606, ..., -0.01607606,
        -0.01607606, -0.01607606],
       [-0.0081806 , -0.0081806 , -0.0081806 , ..., -0.0081806 ,
        -0.0081806 , -0.0081806 ]])

In [ ]:
U, sigma, Vt = svds(matrix_tag_mean, k = 12)

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(6343, 12)
(12,)
(12, 50852)


In [ ]:
sigma = np.diag(sigma)

In [ ]:
sigma[0]

array([131.51684803,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [ ]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_pivot.columns)
df_svd_preds.head()

wine_id,10,19,20,41,46,91,93,114,115,116,127,128,130,135,138,183,198,204,255,271,287,288,296,315,342,344,375,423,433,450,451,452,453,455,457,460,462,473,510,531,...,8003932,8010383,8018306,8018733,8057324,8057883,8075591,8075594,8083851,8095543,8103001,8125365,8127729,8139168,8143180,8152983,8168002,8176779,8203097,8205857,8208422,8226290,8257837,8295247,8295713,8305831,8316340,8331658,8332127,8341431,8342417,8346518,8346599,8357660,8380143,8380239,8407279,8436727,8546714,8564957
0,0.000629,0.013313,0.010222,0.003155,0.033971,0.009674,0.015896,0.016411,0.012794,0.024439,0.054706,0.074203,0.004543,0.025423,0.002110,0.005864,0.007900,0.005813,0.011538,0.005508,0.079783,0.022491,0.060862,0.010121,0.006918,0.024787,0.016465,0.020478,0.009542,0.067261,0.024666,0.001608,-0.000100,0.014361,0.008850,0.002322,0.016033,-0.001205,0.009362,0.047019,...,0.015133,0.030766,0.004192,-0.001122,0.009568,0.013678,0.014203,0.062658,0.006872,0.014728,0.010325,0.008188,0.008506,0.008717,0.000732,-0.009892,0.017352,0.020685,0.008223,-0.007324,0.001223,-0.014604,0.001504,-0.004259,-0.000013,0.019316,-0.001221,0.015909,-0.006681,0.010923,0.006653,-0.007201,0.007283,-0.006154,-0.004298,0.001195,0.009568,0.006000,0.002102,-0.025113
1,0.000177,-0.000615,-0.000349,0.000188,-0.000678,0.000151,0.000384,-0.000513,-0.001506,-0.005627,0.001984,0.004018,0.000007,0.000970,-0.001947,-0.000568,0.001235,-0.001134,0.000161,0.000345,0.002147,0.003370,0.000556,0.000083,-0.000235,0.002575,-0.001786,0.001007,0.000700,-0.003597,0.002087,-0.000043,-0.000633,0.001039,0.003574,-0.000470,0.000699,-0.000625,-0.000432,0.005239,...,-0.000919,0.001000,-0.000066,0.000023,0.000853,-0.000347,0.003040,0.008106,0.000066,0.001065,0.000048,0.000011,-0.000245,-0.000092,0.000445,-0.000736,0.000141,0.000205,0.000415,-0.000378,-0.000874,-0.002241,0.000383,0.000310,-0.000276,-0.000328,-0.000908,-0.000260,-0.000077,0.000548,-0.000062,-0.000383,0.000851,-0.000971,-0.000240,-0.000255,0.000766,-0.000004,-0.000123,-0.001051
2,0.000890,-0.000828,-0.000612,0.000883,-0.002125,0.000610,0.000716,-0.002834,0.000153,-0.006549,0.003478,0.004888,0.000521,0.001761,-0.002666,0.002900,0.002286,-0.000605,0.000359,0.000831,0.001802,0.006010,-0.002497,0.000213,0.000051,0.005194,-0.005469,0.002845,0.001510,-0.005265,-0.002266,-0.000745,-0.001184,0.001360,0.003274,0.000006,0.000415,-0.000938,-0.001719,0.004860,...,-0.001432,0.000595,-0.000072,0.000817,-0.000101,-0.000395,0.007355,0.013249,0.000374,-0.000782,-0.000272,0.001255,0.000228,0.000125,0.000663,-0.001052,-0.000045,0.000205,0.000759,0.002271,-0.002377,-0.002805,0.000881,0.000545,-0.000039,-0.000565,-0.000643,-0.000118,0.002438,0.001508,-0.001569,-0.000573,0.001785,-0.000458,0.001448,0.000171,0.000173,-0.000968,0.000714,0.000988
3,0.009717,-0.007237,-0.005317,0.006396,0.001403,0.003588,-0.003250,0.003377,-0.002363,0.013642,-0.017288,0.028198,0.003603,-0.000955,-0.002140,0.010389,0.018394,-0.001886,0.005711,0.021881,0.016847,0.026030,0.025090,0.005685,0.001377,0.011904,0.010086,0.009302,0.004245,-0.033716,0.113965,0.023934,0.009945,0.007322,0.066202,-0.000682,0.011862,0.009531,0.008247,0.031008,...,-0.003978,-0.005288,0.003592,0.007186,0.005350,-0.000873,-0.002126,0.000588,0.002296,0.012644,0.015614,0.000854,0.002639,0.007309,0.008966,0.012667,0.000698,-0.002492,0.002343,0.005877,0.012011,0.007690,0.013086,0.016086,0.002869,0.000499,0.002238,-0.000885,-0.001810,0.002474,0.008275,0.021891,0.013087,0.003080,0.005742,0.008447,0.004675,0.006281,0.006302,0.025674
4,0.007900,0.001066,0.001342,0.001697,-0.004598,0.000244,0.002102,0.005287,0.014742,0.020832,0.014900,0.029082,0.002754,0.006518,0.006866,0.019627,-0.003862,0.007721,0.002758,-0.006788,0.011287,0.003960,0.014415,-0.000226,0.000683,0.001949,0.002462,0.004234,0.002634,0.017013,0.014855,0.007574,0.003308,-0.001767,-0.006446,0.003862,0.012313,-0.001242,0.003406,-0.017831,...,0.001560,0.020804,-0.000422,0.006439,0.007176,-0.001243,0.001037,-0.011267,0.002495,-0.001426,0.006098,0.008323,0.004553,0.014441,-0.0

In [ ]:
df_pivot.index

Int64Index([    1201,     2764,     2997,     3921,     7078,     7084,
               12420,    16665,    18158,    20226,
            ...
            44298458, 44312405, 44852592, 45350476, 45612185, 46021938,
            46231365, 46356698, 46375247, 46404017],
           dtype='int64', name='userID', length=6343)

In [ ]:
df_svd_preds.index = df_pivot.index

In [ ]:
df_svd_preds

wine_id,10,19,20,41,46,91,93,114,115,116,127,128,130,135,138,183,198,204,255,271,287,288,296,315,342,344,375,423,433,450,451,452,453,455,457,460,462,473,510,531,...,8003932,8010383,8018306,8018733,8057324,8057883,8075591,8075594,8083851,8095543,8103001,8125365,8127729,8139168,8143180,8152983,8168002,8176779,8203097,8205857,8208422,8226290,8257837,8295247,8295713,8305831,8316340,8331658,8332127,8341431,8342417,8346518,8346599,8357660,8380143,8380239,8407279,8436727,8546714,8564957
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1201,0.000629,0.013313,0.010222,0.003155,0.033971,0.009674,0.015896,0.016411,0.012794,0.024439,0.054706,0.074203,0.004543,0.025423,0.002110,0.005864,0.007900,0.005813,0.011538,0.005508,0.079783,0.022491,0.060862,0.010121,0.006918,0.024787,0.016465,0.020478,0.009542,0.067261,0.024666,0.001608,-0.000100,0.014361,0.008850,0.002322,0.016033,-0.001205,0.009362,0.047019,...,0.015133,0.030766,0.004192,-0.001122,0.009568,0.013678,0.014203,0.062658,0.006872,0.014728,0.010325,0.008188,0.008506,0.008717,0.000732,-0.009892,0.017352,0.020685,0.008223,-0.007324,0.001223,-0.014604,0.001504,-0.004259,-0.000013,0.019316,-0.001221,0.015909,-0.006681,0.010923,0.006653,-0.007201,0.007283,-0.006154,-0.004298,0.001195,0.009568,0.006000,0.002102,-0.025113
2764,0.000177,-0.000615,-0.000349,0.000188,-0.000678,0.000151,0.000384,-0.000513,-0.001506,-0.005627,0.001984,0.004018,0.000007,0.000970,-0.001947,-0.000568,0.001235,-0.001134,0.000161,0.000345,0.002147,0.003370,0.000556,0.000083,-0.000235,0.002575,-0.001786,0.001007,0.000700,-0.003597,0.002087,-0.000043,-0.000633,0.001039,0.003574,-0.000470,0.000699,-0.000625,-0.000432,0.005239,...,-0.000919,0.001000,-0.000066,0.000023,0.000853,-0.000347,0.003040,0.008106,0.000066,0.001065,0.000048,0.000011,-0.000245,-0.000092,0.000445,-0.000736,0.000141,0.000205,0.000415,-0.000378,-0.000874,-0.002241,0.000383,0.000310,-0.000276,-0.000328,-0.000908,-0.000260,-0.000077,0.000548,-0.000062,-0.000383,0.000851,-0.000971,-0.000240,-0.000255,0.000766,-0.000004,-0.000123,-0.001051
2997,0.000890,-0.000828,-0.000612,0.000883,-0.002125,0.000610,0.000716,-0.002834,0.000153,-0.006549,0.003478,0.004888,0.000521,0.001761,-0.002666,0.002900,0.002286,-0.000605,0.000359,0.000831,0.001802,0.006010,-0.002497,0.000213,0.000051,0.005194,-0.005469,0.002845,0.001510,-0.005265,-0.002266,-0.000745,-0.001184,0.001360,0.003274,0.000006,0.000415,-0.000938,-0.001719,0.004860,...,-0.001432,0.000595,-0.000072,0.000817,-0.000101,-0.000395,0.007355,0.013249,0.000374,-0.000782,-0.000272,0.001255,0.000228,0.000125,0.000663,-0.001052,-0.000045,0.000205,0.000759,0.002271,-0.002377,-0.002805,0.000881,0.000545,-0.000039,-0.000565,-0.000643,-0.000118,0.002438,0.001508,-0.001569,-0.000573,0.001785,-0.000458,0.001448,0.000171,0.000173,-0.000968,0.000714,0.000988
3921,0.009717,-0.007237,-0.005317,0.006396,0.001403,0.003588,-0.003250,0.003377,-0.002363,0.013642,-0.017288,0.028198,0.003603,-0.000955,-0.002140,0.010389,0.018394,-0.001886,0.005711,0.021881,0.016847,0.026030,0.025090,0.005685,0.001377,0.011904,0.010086,0.009302,0.004245,-0.033716,0.113965,0.023934,0.009945,0.007322,0.066202,-0.000682,0.011862,0.009531,0.008247,0.031008,...,-0.003978,-0.005288,0.003592,0.007186,0.005350,-0.000873,-0.002126,0.000588,0.002296,0.012644,0.015614,0.000854,0.002639,0.007309,0.008966,0.012667,0.000698,-0.002492,0.002343,0.005877,0.012011,0.007690,0.013086,0.016086,0.002869,0.000499,0.002238,-0.000885,-0.001810,0.002474,0.008275,0.021891,0.013087,0.003080,0.005742,0.008447,0.004675,0.006281,0.006302,0.025674
7078,0.007900,0.001066,0.001342,0.001697,-0.004598,0.000244,0.002102,0.005287,0.014742,0.020832,0.014900,0.029082,0.002754,0.006518,0.006866,0.019627,-0.003862,0.007721,0.002758,-0.006788,0.011287,0.003960,0.014415,-0.000226,0.000683,0.001949,0.002462,0.004234,0.002634,0.017013,0.014855,0.007574,0.003308,-0.001767,-0.006446,0.003862,0.012313,-0.001242,0.003406,-0.017831,...,0.001560,0.020804,-0.000422,0.0

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
hit = 0
total = 0
results = []
for user_id in tqdm(df_svd_preds.index):
  predictions = df_svd_preds.loc[user_id, :].sort_values(ascending=False).head(500).index.values
  trues = df_test.loc[(df_test.userID == user_id) & (df_test.like == 1),'wine_id'].values
  correct = np.intersect1d(predictions, trues)
  total += trues.shape[0]
  hit += correct.shape[0]
  results.append((trues.shape[0], correct.shape[0]))

In [ ]:
df_result = pd.DataFrame(results, columns=['trues', 'hit'])

In [ ]:
df_result['recall'] = df_result.hit / df_result.trues
df_result['precision'] = df_result.hit / 500

In [ ]:
hit, total, hit/total

(33929, 100491, 0.3376322257714621)

In [ ]:
df_result.mean()

trues        15.842819
hit           5.349046
recall        0.476846
precision     0.010698
dtype: float64

In [ ]:
df_result

,hit,trues,precision,recall
0,19,13,0.038,1.461538
1,1,1,0.002,1.000000
2,1,1,0.002,1.000000
3,14,2,0.028,7.000000
4,7,2,0.014,3.500000
...,...,...,...,...
6338,9,2,0.018,4.500000
6339,30,16,0.060,1.875000
6340,1,1,0.002,1.000000
6341,29,10,0.058,2.900000


## 2. ROC 구하기 (like)

In [7]:
df_pivot = df_train.pivot_table('like', 'userID', 'wine_id', aggfunc='max').fillna(0)

In [8]:
matrix = df_pivot.values
user_ratings_mean = np.mean(matrix, axis = 1)
# R_user_mean : 사용자-와인에 대해 사용자 평균 평점을 뺀 것.
matrix_tag_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [9]:
U, sigma, Vt = svds(matrix_tag_mean, k = 12)
sigma = np.diag(sigma)

In [10]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [11]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_pivot.columns)
df_svd_preds.head()

wine_id,10,19,20,41,46,91,93,114,115,116,127,128,130,135,138,183,198,204,255,271,287,288,296,315,342,344,375,423,433,450,451,452,453,455,457,460,462,473,510,531,...,8003932,8010383,8018306,8018733,8057324,8057883,8075591,8075594,8083851,8095543,8103001,8125365,8127729,8139168,8143180,8152983,8168002,8176779,8203097,8205857,8208422,8226290,8257837,8295247,8295713,8305831,8316340,8331658,8332127,8341431,8342417,8346518,8346599,8357660,8380143,8380239,8407279,8436727,8546714,8564957
0,-0.001430,0.002847,0.001980,0.000499,0.003516,0.000056,4.552684e-04,0.001396,-0.002504,-0.001037,0.006179,0.003065,4.552684e-04,0.002139,-0.000107,0.000491,0.000310,0.000162,0.000953,-0.001498,-0.002175,0.003477,-0.001862,0.000278,0.000340,0.001672,-0.002217,0.002076,0.000866,0.006054,-0.004330,-0.000760,-0.000033,0.000812,-0.002857,-3.392214e-04,0.000605,0.000071,0.000446,0.007903,...,-0.001610,0.003703,-0.000460,0.000349,9.398824e-04,0.001870,0.006085,0.015373,-0.000081,0.000533,0.001018,-0.000176,0.001086,-0.000328,0.000697,0.000036,0.001653,0.000519,0.000606,-0.000320,0.000590,-0.001873,-0.000616,4.552684e-04,0.000293,0.001971,-0.000060,1.272439e-03,-0.000500,0.000782,0.001930,0.001985,0.001149,-0.000301,-0.001463,0.000412,0.000795,0.001259,2.676021e-04,-0.001105
1,-0.000022,-0.000046,-0.000045,0.000004,-0.000070,0.000003,-2.200551e-07,-0.000035,-0.000053,-0.000068,0.000057,-0.000036,-2.200551e-07,0.000007,-0.000014,-0.000003,-0.000010,-0.000039,-0.000013,-0.000065,-0.000082,0.000037,-0.000169,-0.000003,-0.000030,0.000082,-0.000076,0.000030,0.000018,-0.000201,-0.000085,-0.000018,-0.000012,-0.000002,0.000073,-2.115742e-07,-0.000020,-0.000016,-0.000008,-0.000002,...,-0.000078,0.000017,-0.000014,0.000047,1.898816e-07,-0.000041,0.000114,0.000332,0.000006,0.000012,0.000040,-0.000024,-0.000012,0.000002,0.000004,0.000014,0.000035,0.000023,0.000007,0.000060,0.000015,-0.000051,-0.000004,-2.200551e-07,0.000009,-0.000047,-0.000005,5.102744e-07,0.000022,0.000010,0.000016,0.000007,0.000032,0.000008,0.000052,-0.000001,0.000017,0.000013,-3.416149e-08,0.000025
2,-0.000175,0.000037,-0.000173,0.000107,-0.000406,-0.000044,3.026088e-05,-0.000354,-0.000799,-0.000953,0.000723,0.000620,3.026088e-05,0.000003,-0.000203,-0.000106,-0.000032,-0.000118,-0.000196,-0.000401,-0.001029,0.000201,-0.001457,-0.000027,-0.000140,0.000510,-0.000514,0.000329,0.000071,-0.001574,-0.001195,-0.000121,-0.000130,0.000089,0.000007,2.988237e-05,0.000102,-0.000096,-0.000127,0.001352,...,-0.000581,-0.000054,-0.000130,0.000216,-1.706623e-04,0.000031,0.000827,0.002094,0.000034,0.000001,0.000455,0.000159,0.000022,-0.000141,0.000064,0.000042,0.000266,-0.000138,0.000177,0.000552,-0.000066,-0.000161,-0.000025,3.026088e-05,0.000038,-0.000084,-0.000163,1.159673e-04,0.000129,0.000171,-0.000169,-0.000160,0.000279,-0.000078,0.000309,-0.000022,0.000095,-0.000077,1.001945e-04,0.000415
3,-0.000232,-0.000710,-0.000683,0.000106,0.000746,-0.000033,-4.120535e-04,0.000235,0.000735,0.002690,-0.000695,0.001758,-4.120535e-04,0.000754,-0.000134,0.001721,0.000804,0.000284,-0.000074,0.001175,0.002995,-0.000105,0.004044,-0.000501,0.000615,0.000280,0.000960,-0.000035,-0.000185,0.000243,0.002826,0.001985,0.000363,-0.000225,0.001882,-1.794350e-04,0.000032,0.000397,0.000389,0.001142,...,0.000260,-0.000157,-0.000068,-0.000347,-7.191387e-04,0.000972,-0.001210,-0.001772,-0.000047,-0.000152,0.000320,0.000444,0.000994,0.000048,-0.000205,-0.000201,-0.000412,0.000143,-0.000163,0.000311,0.000012,0.000534,0.000210,-4.120535e-04,-0.000331,0.001547,0.000391,3.661951e-05,-0.000568,-0.000157,-0.000342,-0.000691,0.000467,-0.000007,0.000071,-0.000213,-0.000474,-0.000373,-1.732910e-04,0.000538
4,0.000588,0.000118,0.000213,0.000345,-0.000360,0.000144,4.465085e-04,0.000342,0.001098,0.000234,0.001768,0.002345,4.465085e-04,0.000400,0.000034,0.001886,-0.000134,0.000200,0.000162,-0.000018,-0.000139,-0.000311,0.000422,0.000354,0.000259,0.000193,0.000132,0.000101,0.000402,0.000939,0.000512,0.000830,0.000601,0.000383,-0.000428,3.591942e-04,0.000664

In [12]:
df_svd_preds.index = df_pivot.index

In [13]:
from tqdm.notebook import tqdm

In [16]:
df_svd_preds.loc[1201,:].sort_values(ascending=False)

wine_id
1152755    0.987424
1127795    0.973904
1684223    0.973047
1655970    0.877677
3908       0.819094
             ...   
4086      -0.108296
66284     -0.147772
1128385   -0.174410
1135069   -0.206087
1911534   -0.253678
Name: 1201, Length: 50852, dtype: float64

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

def get_result(model):
  y_pred = model.predict(cached_test).flatten()
  y_pred_class = [1 if pred > 0.5 else 0 for pred in y_pred]

  print(f"ROC: {roc_auc_score(y_true, y_pred)}")
  print(classification_report(y_true, y_pred_class))

In [18]:
df_test[['userID', 'wine_id','like']]

,userID,wine_id,like
0,1201,1214547,0
1,1201,1099253,0
2,1201,1684223,0
3,1201,100044,1
4,1201,1107909,1
...,...,...,...
188713,46404017,1155203,1
188714,46404017,4382344,1
188715,46404017,1104850,0
188716,46404017,1981888,0


In [25]:
predictions = [] 
for user_id, wine_id in zip(df_test.userID, df_test.wine_id):
  try:
    predictions.append(df_svd_preds.loc[user_id, wine_id])
  except:
    predictions.append(None)

In [26]:
df_test['prediction'] = predictions

In [29]:
df_test2 = df_test[['like', 'prediction']]

In [32]:
df_test2 = df_test2.dropna().reset_index(drop=True)

In [33]:
df_test2

,like,prediction
0,0,0.060100
1,0,0.046618
2,0,0.973047
3,1,0.025583
4,1,0.015133
...,...,...
188709,1,0.011669
188710,1,0.005689
188711,0,0.000010
188712,0,0.000798


In [ ]:
df_test2

In [34]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

# def get_result(model):
#   y_pred = model.predict(cached_test).flatten()
#   y_pred_class = [1 if pred > 0.5 else 0 for pred in y_pred]

#   print(f"ROC: {roc_auc_score(y_true, y_pred)}")
#   print(classification_report(y_true, y_pred_class))

In [35]:
roc_auc_score(df_test2.like, df_test2.prediction)

0.7060730406522705